In [ ]:
import cv2
import numpy as np
import time
import json
import urllib.request

cap = cv2.VideoCapture(0)

if (cap.isOpened() == False): 
    print("Unable to read camera feed")

turn_url = "http://pub.inspace.co.kr:10200/turn_status"
action_url = "http://pub.inspace.co.kr:10200/action?action_idx="
end_url =  "http://pub.inspace.co.kr:10200/periodic_status"

while(1):

    end_response = urllib.request.urlopen(end_url)
    end_result = json.loads(end_response.read().decode('utf-8'))
    
    if end_result['win_index'] == 0 : #End Check
        is_End = False
        is_Turn = True
    
    while not is_End:
        
        turn_response = urllib.request.urlopen(turn_url)
        turn_result = json.loads(turn_response.read().decode('utf-8'))

        if turn_result['curr_turn'] == turn_result['enemy_turn'] : #Turn Check
            # Not player turn
            time.sleep(1)

        else:
            # Player turn
            is_Change = False

            # time delay or stable check

            ret, frame = cap.read()
            pre_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            while not is_Change:
                frame_list = []

                is_Stable = False

                while not is_Stable:

                    ret, frame = cap.read()
                    cur_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                    if ret == True :
                        cv2.imshow('cur_frame', cur_frame)
                        # time.sleep(1)

                        if cv2.waitKey(1) & 0xFF == ord('q'):
                            break
                        if len(frame_list):
                            frame_list = np.append(frame_list,np.array([cur_frame]),axis=0)
                        else:
                            frame_list = np.array([cur_frame])

                        if len(frame_list) == 5 :

                            # Stable Check
                            if np.max(np.std(frame_list,axis=0)) < 50:
                                is_Stable = True
                                print('Stable!')

                            else:
                                print(np.max(np.std(frame_list,axis=0)))
                                frame_list = frame_list[1:]
                                print('Not Stable!')
                    else:
                        break
                # Change Check

                if np.mean(abs(pre_frame - frame_list[-1])) > 10:
                    is_Change = True
                    check_frame = abs(pre_frame - frame_list[-1])
                    cv2.imshow('check_frame', check_frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    is_Change = True
                    
                    output_idx = 0
                    
                    action_response = urllib.request.urlopen(action_url + str(output_idx))
                    action_result = json.loads(action_response.read().decode('utf-8'))
                    

                    print('Change!')
                else:
                    print('Not Change!')
